In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split # Para dividir claves de series
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf # Importar tensorflow
import os
import joblib
import gc

# --- Parámetros Configurables (igual que antes) ---
FILE_PATH = '/content/drive/MyDrive/TFM/data/gold_venta_semanales_clusters.parquet' # ¡CAMBIA ESTO!
LOOK_BACK = 26  # Número de semanas anteriores a usar para predecir la siguiente
EPOCHS = 50
BATCH_SIZE = 32
MODEL_SAVE_DIR = '/content/drive/MyDrive/TFM/modelos_gru_por_cluster_final'

# Asegúrate de que FEATURE_COLS y TARGET_COL están definidos
FEATURE_COLS = ['weekly_volume', 'has_promo', 'is_covid_period']
TARGET_COL = 'weekly_volume'


os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# --- 1. Cargar Datos ---
try:
    df = pd.read_parquet(FILE_PATH)
except Exception as e:
    print(f"Error cargando el archivo Parquet: {e}")
    exit()

df = df.sort_values(by=['cluster_label', 'establecimiento', 'material', 'week'])

# --- Función generadora de secuencias para tf.data.Dataset ---
# Esta función generará secuencias para un conjunto de claves de series
def series_sequence_generator(list_of_series_keys, scaled_data_cache_local, look_back_local):
    for key in list_of_series_keys: # key es una tupla (establecimiento, material)
        ts_scaled_ordered_local = scaled_data_cache_local.get(key) # Usar .get para evitar KeyError si falta
        if ts_scaled_ordered_local is None or len(ts_scaled_ordered_local) <= look_back_local:
            continue

        # Crear secuencias para esta única serie
        for i in range(len(ts_scaled_ordered_local) - look_back_local):
            X_sample = ts_scaled_ordered_local[i:(i + look_back_local)]
            # El target es la última columna del array escalado en el paso i + look_back_local
            y_sample = ts_scaled_ordered_local[i + look_back_local, -1]
            yield X_sample, y_sample

# --- 3. Iterar por Clúster ---
unique_clusters = df['cluster_label'].unique()
all_trained_models = {}
all_scalers_info = {} # Para guardar rutas a los scalers

for cluster_id in unique_clusters:
    print(f"\n--- Procesando Cluster: {cluster_id} ---")
    df_cluster = df[df['cluster_label'] == cluster_id].copy() # .copy() es más seguro aquí

    if df_cluster.empty:
        print(f"Cluster {cluster_id} no tiene datos. Saltando.")
        del df_cluster
        gc.collect()
        continue

    # Cache para datos escalados y scalers para el cluster actual
    scaled_series_data_cache = {}
    scalers_cluster_ts = {} # Para guardar los scalers antes de persistirlos

    # Preparar datos y scalers para todas las series en el clúster
    all_series_keys_in_cluster = []
    num_features = None

    # Paso 1: Iterar para ajustar scalers y preparar datos escalados
    for (establecimiento, material), group in df_cluster.groupby(['establecimiento', 'material']):
        series_key = (establecimiento, material)
        all_series_keys_in_cluster.append(series_key)

        if len(group) <= LOOK_BACK:
            print(f"  Datos insuficientes para la serie {series_key} ({len(group)} puntos). Necesita > {LOOK_BACK}.")
            continue

        # Reordenar columnas para que TARGET_COL sea la última
        current_feature_cols_ordered = [col for col in FEATURE_COLS if col != TARGET_COL] + [TARGET_COL]
        ts_features_ordered = group[current_feature_cols_ordered].values.astype('float32')

        # Escalar todas las features ordenadas juntas
        feature_scaler_ordered = MinMaxScaler(feature_range=(0, 1))
        ts_scaled_ordered = feature_scaler_ordered.fit_transform(ts_features_ordered)

        # Scaler específico para el target (para la inversión)
        target_scaler = MinMaxScaler(feature_range=(0, 1))
        target_scaler.fit(group[[TARGET_COL]].values.astype('float32'))

        scalers_cluster_ts[series_key] = {
            'features_scaler_ordered': feature_scaler_ordered,
            'target_scaler': target_scaler
        }
        scaled_series_data_cache[series_key] = ts_scaled_ordered

        if num_features is None: # Determinar el número de features del primer set de datos válido
            num_features = ts_scaled_ordered.shape[1]

    if num_features is None:
        print(f"Cluster {cluster_id}: No se encontraron features válidas en ninguna serie. Saltando clúster.")
        del df_cluster, scaled_series_data_cache, scalers_cluster_ts
        gc.collect()
        continue

    # Guardar los scalers para el clúster actual
    scalers_path = os.path.join(MODEL_SAVE_DIR, f'scalers_cluster_{cluster_id}.joblib')
    joblib.dump(scalers_cluster_ts, scalers_path)
    all_scalers_info[cluster_id] = scalers_path
    print(f"  Scalers para clúster {cluster_id} guardados en: {scalers_path}")


    # Paso 2: Dividir las CLAVES de las series en entrenamiento y prueba
    if not all_series_keys_in_cluster:
        print(f"Cluster {cluster_id}: No hay series con suficientes datos. Saltando entrenamiento.")
        del df_cluster, scaled_series_data_cache, scalers_cluster_ts # Limpiar
        gc.collect()
        continue

    train_series_keys, test_series_keys = train_test_split(all_series_keys_in_cluster, test_size=0.2, random_state=42)

    if not train_series_keys:
        print(f"Cluster {cluster_id}: No hay series asignadas para entrenamiento después del split. Saltando.")
        del df_cluster, scaled_series_data_cache, scalers_cluster_ts # Limpiar
        gc.collect()
        continue

    # Paso 3: Crear tf.data.Dataset usando los generadores
    output_signature = (
        tf.TensorSpec(shape=(LOOK_BACK, num_features), dtype=tf.float32), # X: (look_back, num_features)
        tf.TensorSpec(shape=(), dtype=tf.float32)                      # y: escalar
    )

    train_dataset = tf.data.Dataset.from_generator(
        lambda: series_sequence_generator(train_series_keys, scaled_series_data_cache, LOOK_BACK),
        output_signature=output_signature
    )

    # Contar secuencias totales para el buffer de shuffle (opcional pero bueno para el rendimiento)
    total_train_sequences = sum(max(0, len(scaled_series_data_cache[key]) - LOOK_BACK) for key in train_series_keys if key in scaled_series_data_cache)

    if total_train_sequences == 0:
        print(f"Cluster {cluster_id}: No se generaron secuencias de entrenamiento. Saltando entrenamiento del modelo.")
        del df_cluster, scaled_series_data_cache, scalers_cluster_ts, train_dataset
        gc.collect()
        continue

    shuffle_buffer_size = min(total_train_sequences, 5000) # Ajusta este valor según tu memoria
    train_dataset = train_dataset.shuffle(shuffle_buffer_size).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Crear dataset de prueba (validación)
    if test_series_keys:
        test_dataset = tf.data.Dataset.from_generator(
            lambda: series_sequence_generator(test_series_keys, scaled_series_data_cache, LOOK_BACK),
            output_signature=output_signature
        )
        test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    else:
        test_dataset = None
        print(f"Cluster {cluster_id}: No hay series para el conjunto de validación.")


    # --- Construir Modelo GRU ---
    # (La input_shape ahora se infiere o se puede pasar num_features directamente)
    model = Sequential([
        GRU(units=64, input_shape=(LOOK_BACK, num_features), return_sequences=True),
        Dropout(0.2),
        GRU(units=32, return_sequences=False),
        Dropout(0.2),
        Dense(units=16, activation='relu'),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    # model.summary() # Descomentar si quieres ver el resumen

    # --- Entrenar Modelo ---
    print(f"  Entrenando modelo para el clúster {cluster_id}...")
    early_stopping = EarlyStopping(monitor='val_loss' if test_dataset else 'loss', patience=10, restore_best_weights=True)

    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=test_dataset, # Puede ser None
        callbacks=[early_stopping],
        verbose=1
    )

    # --- Almacenar Modelo ---
    model_path = os.path.join(MODEL_SAVE_DIR, f'gru_model_cluster_{cluster_id}.keras')
    model.save(model_path)
    all_trained_models[cluster_id] = model_path
    print(f"  Modelo para clúster {cluster_id} guardado en: {model_path}")

    if test_dataset:
        loss = model.evaluate(test_dataset, verbose=0)
        print(f"  Pérdida (MSE) en datos de prueba para clúster {cluster_id}: {loss:.4f}")

    # Limpiar memoria del clúster actual antes de pasar al siguiente
    del df_cluster, scaled_series_data_cache, scalers_cluster_ts
    del train_dataset, test_dataset, model, history, train_series_keys, test_series_keys, all_series_keys_in_cluster
    gc.collect()


print("\n--- Entrenamiento Completado ---")
print("Modelos guardados:", all_trained_models)
print("Rutas a Scalers guardados:", all_scalers_info)

# --- La función de predicción necesitaría adaptarse ligeramente ---
# El cargado de scalers ahora es desde 'all_scalers_info' que contiene rutas a archivos joblib
# que a su vez contienen diccionarios de scalers por (establecimiento, material) para ese clúster.
# ... (La función predict_volume necesitará cargar el archivo joblib del cluster_id_pred)
# Ejemplo de modificación en predict_volume:
# scalers_cluster_file = all_scalers_info.get(cluster_id_pred) # O construir la ruta como antes
# if scalers_cluster_file is None or not os.path.exists(scalers_cluster_file): ...
# loaded_scalers_all_ts_in_cluster = joblib.load(scalers_cluster_file)
# feature_scaler_ordered = loaded_scalers_all_ts_in_cluster[ts_key]['features_scaler_ordered']
# target_scaler_pred = loaded_scalers_all_ts_in_cluster[ts_key]['target_scaler']

Streaming output truncated to the last 5000 lines.
  Datos insuficientes para la serie ('8100426246', 'VI15') (23 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426248', 'ED15LN') (16 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426248', 'ED30') (19 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426253', 'DL13') (12 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426253', 'DL13SR') (13 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426253', 'FD13') (12 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426256', 'DL30') (13 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426258', 'ED30') (24 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426259', 'ED13') (21 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426269', 'EDI13P6') (12 puntos). Necesita > 26.
  Datos insuficientes para la serie ('8100426272', 'DL20') (15 puntos). 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


  Entrenando modelo para el clúster 0...
Epoch 1/50
1251143/Unknown 12588s 10ms/step - loss: 0.0287

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1251144/1251144 ━━━━━━━━━━━━━━━━━━━━ 15347s 12ms/step - loss: 0.0287 - val_loss: 0.0285
Epoch 2/50
1251144/1251144 ━━━━━━━━━━━━━━━━━━━━ 15211s 12ms/step - loss: 0.0301 - val_loss: 0.0391
Epoch 3/50
1251144/1251144 ━━━━━━━━━━━━━━━━━━━━ 15450s 12ms/step - loss: 0.0348 - val_loss: 0.0328
Epoch 4/50
1251144/1251144 ━━━━━━━━━━━━━━━━━━━━ 15232s 12ms/step - loss: 0.0350 - val_loss: 0.0337
Epoch 5/50
1251144/1251144 ━━━━━━━━━━━━━━━━━━━━ 15114s 12ms/step - loss: 0.0362 - val_loss: 0.0343
Epoch 6/50
 782522/1251144 ━━━━━━━━━━━━━━━━━━━━ 1:17:07 10ms/step - loss: 0.0336

In [ ]:
N_TIMESTEPS = 52 # Lookback window
N_TIMESTEPS_STR = str(N_TIMESTEPS)

In [ ]:
import pandas as pd

df = pd.read_parquet('/content/drive/MyDrive/TFM/data/gold_venta_semanales_clusters.parquet')
df.head()

,establecimiento,material,week,has_promo,weekly_volume,is_covid_period,cluster_label
0,8100005624,ED13LT,2017-01-23,0,21954.24,0,1
1,8100005624,ED13LT,2017-01-30,0,0.00,0,1
2,8100005624,ED13LT,2017-02-06,0,0.00,0,1
3,8100005624,ED13LT,2017-02-13,0,0.00,0,1
4,8100005624,ED13LT,2017-02-20,0,0.00,0,1


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Concatenate, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import joblib
import json
import os
import matplotlib.pyplot as plt

# --- 1. Configuración ---

# Directorio donde guardamos los datos preprocesados
DATA_PREP_DIR = os.path.join('/content/drive/MyDrive/TFM/gru/gru_data_preparation_output')
# Directorio donde guardaremos el modelo entrenado y los resultados
MODEL_OUTPUT_DIR = os.path.join('/content/drive/MyDrive/TFM/gru/gru_model_output')
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

# Archivos de entrada (generados por el script anterior)
TRAIN_DATA_FILE = os.path.join(DATA_PREP_DIR, 'gru_train_data.npz')
VAL_DATA_FILE = os.path.join(DATA_PREP_DIR, 'gru_val_data.npz')
MAPPINGS_FILE = os.path.join(DATA_PREP_DIR, 'gru_cat_mappings.json')
# SCALER_FILE = os.path.join(DATA_PREP_DIR, 'gru_scaler.joblib') # Lo cargaremos en la evaluación

# Archivo de salida para el mejor modelo
BEST_MODEL_FILE = os.path.join(MODEL_OUTPUT_DIR, 'best_gru_model.keras') # Usar .keras es el formato preferido

# --- Hiperparámetros del Modelo ---
# Deben coincidir con la preparación de datos
VAL_SIZE_RATIO = 0.15 # 15% para validación
TEST_SIZE_RATIO = 0.15 # 15% para test

# Dimensiones de Embedding (ajustables)
EMBEDDING_DIM_ESTAB = 10
EMBEDDING_DIM_MATERIAL = 5
EMBEDDING_DIM_CLUSTER = 3
# Unidades GRU y Dropout (ajustables)
GRU_UNITS = 64
DROPOUT_RATE = 0.2
# Capa densa oculta (opcional, ajustar unidades o quitarla poniendo None)
DENSE_UNITS = 32

# --- Parámetros de Entrenamiento ---
EPOCHS = 100 # Número máximo de épocas (EarlyStopping decidirá)
BATCH_SIZE = 64
PATIENCE_EARLY_STOPPING = 10 # Paciencia para EarlyStopping
PATIENCE_REDUCE_LR = 5      # Paciencia para reducir Learning Rate


# --- 2. Cargar Datos Preprocesados y Mapeos ---
print("Cargando datos preprocesados y mapeos...")
try:
    # Cargar mapeos para obtener tamaños de vocabulario
    with open(MAPPINGS_FILE, 'r') as f:
        mappings_data = json.load(f)
        vocab_sizes = mappings_data['vocab_sizes']
        # Mapeos no son necesarios aquí, solo los tamaños

    # Cargar datos de entrenamiento
    train_data = np.load(TRAIN_DATA_FILE)
    X_train_num = train_data['X_num']
    y_train = train_data['y']
    # Extraer inputs categóricos (asegúrate que los nombres coinciden con los guardados)
    X_train_cat_estab = train_data['establecimiento_mapped']
    X_train_cat_material = train_data['material_mapped']
    X_train_cat_cluster = train_data['cluster_label_mapped']

    # Cargar datos de validación
    val_data = np.load(VAL_DATA_FILE)
    X_val_num = val_data['X_num']
    y_val = val_data['y']
    # Extraer inputs categóricos
    X_val_cat_estab = val_data['establecimiento_mapped']
    X_val_cat_material = val_data['material_mapped']
    X_val_cat_cluster = val_data['cluster_label_mapped']

    # Obtener dimensiones de los datos cargados
    n_samples_train, n_timesteps_loaded, n_numerical_features = X_train_num.shape
    n_samples_val = X_val_num.shape[0]

    # Validar n_timesteps
    if n_timesteps_loaded != N_TIMESTEPS:
        raise ValueError(f"N_TIMESTEPS en config ({N_TIMESTEPS}) no coincide con datos cargados ({n_timesteps_loaded})")

    print(f"Datos de entrenamiento cargados: X_num={X_train_num.shape}, y={y_train.shape}")
    print(f"  Inputs categóricos train shapes: estab={X_train_cat_estab.shape}, "
          f"material={X_train_cat_material.shape}, cluster={X_train_cat_cluster.shape}")
    print(f"Datos de validación cargados: X_num={X_val_num.shape}, y={y_val.shape}")
    print(f"Tamaños de Vocabulario: {vocab_sizes}")

    # Preparar listas de inputs para model.fit
    X_train_inputs = [X_train_num, X_train_cat_estab, X_train_cat_material, X_train_cat_cluster]
    X_val_inputs = [X_val_num, X_val_cat_estab, X_val_cat_material, X_val_cat_cluster]

except Exception as e:
    print(f"Error cargando datos preprocesados: {e}")
    exit()


# --- 3. Construir el Modelo GRU (API Funcional) ---
print("\nConstruyendo el modelo GRU...")

def build_gru_model(n_timesteps, n_num_features, vocab_sizes_dict, emb_dims_dict, gru_units, dropout_rate, dense_units=None):
    """Construye el modelo GRU multi-input."""

    # Inputs
    input_numerical = Input(shape=(n_timesteps, n_num_features), name='numerical_input')
    input_cat_estab = Input(shape=(1,), name='establecimiento_input', dtype='int32')
    input_cat_material = Input(shape=(1,), name='material_input', dtype='int32')
    input_cat_cluster = Input(shape=(1,), name='cluster_input', dtype='int32')

    # Embeddings
    emb_estab = Embedding(input_dim=vocab_sizes_dict['establecimiento'], output_dim=emb_dims_dict['establecimiento'], name='establecimiento_embedding')(input_cat_estab)
    emb_material = Embedding(input_dim=vocab_sizes_dict['material'], output_dim=emb_dims_dict['material'], name='material_embedding')(input_cat_material)
    emb_cluster = Embedding(input_dim=vocab_sizes_dict['cluster_label'], output_dim=emb_dims_dict['cluster_label'], name='cluster_embedding')(input_cat_cluster)

    flat_emb_estab = Flatten()(emb_estab)
    flat_emb_material = Flatten()(emb_material)
    flat_emb_cluster = Flatten()(emb_cluster)

    # GRU
    gru_layer = GRU(units=gru_units, name='gru_layer')(input_numerical)
    gru_layer = Dropout(dropout_rate)(gru_layer)

    # Concatenate
    combined = Concatenate(name='concatenate_features')([gru_layer, flat_emb_estab, flat_emb_material, flat_emb_cluster])

    # Dense layers
    x = combined
    if dense_units:
      x = Dense(dense_units, activation='relu', name='hidden_dense')(x)
      # x = Dropout(dropout_rate)(x) # Opcional: Dropout también aquí

    output = Dense(1, activation='linear', name='output')(x) # Salida de regresión

    # Crear Modelo
    model = Model(inputs=[input_numerical, input_cat_estab, input_cat_material, input_cat_cluster], outputs=output)
    return model

# Crear instancia del modelo
model = build_gru_model(
    n_timesteps=N_TIMESTEPS,
    n_num_features=n_numerical_features,
    vocab_sizes_dict=vocab_sizes,
    emb_dims_dict={
        'establecimiento': EMBEDDING_DIM_ESTAB,
        'material': EMBEDDING_DIM_MATERIAL,
        'cluster_label': EMBEDDING_DIM_CLUSTER
    },
    gru_units=GRU_UNITS,
    dropout_rate=DROPOUT_RATE,
    dense_units=DENSE_UNITS
)

# Imprimir resumen del modelo
model.summary()


# --- 4. Compilar el Modelo ---
print("\nCompilando el modelo...")
model.compile(optimizer='adam', loss='mse', metrics=['mae']) # Mean Squared Error para regresión


# --- 5. Configurar Callbacks ---
print("Configurando callbacks...")

# Detener si la pérdida de validación no mejora
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE_EARLY_STOPPING,
    restore_best_weights=True # Restaurar pesos del mejor epoch
)

# Guardar solo el mejor modelo basado en la pérdida de validación
model_checkpoint = ModelCheckpoint(
    filepath=BEST_MODEL_FILE,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False # Guardar arquitectura completa + pesos
)

# Reducir learning rate si el entrenamiento se estanca
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, # Nuevo lr = lr * factor
    patience=PATIENCE_REDUCE_LR,
    min_lr=1e-6 # Límite inferior para el learning rate
)

callbacks_list = [early_stopping, model_checkpoint, reduce_lr]


# --- 6. Entrenar el Modelo ---
print("\nIniciando entrenamiento del modelo...")

history = model.fit(
    X_train_inputs,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_inputs, y_val),
    callbacks=callbacks_list,
    verbose=1 # Mostrar barra de progreso
)

print("\n--- Entrenamiento completado ---")
print(f"El mejor modelo ha sido guardado en: {BEST_MODEL_FILE}")


# --- 7. (Opcional) Visualizar Historial de Entrenamiento ---
print("\nGenerando gráfico de historial de entrenamiento...")

def plot_training_history(history, output_dir):
    """Genera gráficos de pérdida y métricas de entrenamiento/validación."""
    try:
        fig, ax1 = plt.subplots(figsize=(12, 6))

        # Plot Loss
        color = 'tab:red'
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss (MSE)', color=color)
        ax1.plot(history.history['loss'], label='Train Loss', color=color, linestyle='-')
        ax1.plot(history.history['val_loss'], label='Validation Loss', color=color, linestyle='--')
        ax1.tick_params(axis='y', labelcolor=color)
        ax1.legend(loc='upper left')
        ax1.grid(True)

        # Instanciar un segundo eje Y para MAE
        ax2 = ax1.twinx()
        color = 'tab:blue'
        ax2.set_ylabel('MAE', color=color)
        ax2.plot(history.history['mae'], label='Train MAE', color=color, linestyle='-')
        ax2.plot(history.history['val_mae'], label='Validation MAE', color=color, linestyle='--')
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.legend(loc='upper right')

        fig.tight_layout() # Ajustar layout
        plt.title('Training & Validation Loss/MAE per Epoch')
        plot_filename = os.path.join(output_dir, 'training_history.png')
        plt.savefig(plot_filename)
        plt.close() # Cerrar figura
        print(f"Gráfico de historial guardado en: {plot_filename}")
    except Exception as e:
        print(f"Error al generar gráfico de historial: {e}")

plot_training_history(history, MODEL_OUTPUT_DIR)

print("\nScript de entrenamiento finalizado.")

Cargando datos preprocesados y mapeos...
Error cargando datos preprocesados: [Errno 2] No such file or directory: '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_/52/gru_cat_mappings.json'

Construyendo el modelo GRU...


NameError: name 'n_numerical_features' is not defined

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import joblib
import json
import os
import pandas as pd # Necesario para recargar datos originales para MASE
import matplotlib.pyplot as plt

# --- 1. Configuración ---

# Directorio donde guardamos los datos preprocesados
DATA_PREP_DIR = os.path.join('/content/drive/MyDrive/TFM/gru/gru_data_preparation_output')
# Directorio donde guardamos el modelo entrenado
MODEL_OUTPUT_DIR = os.path.join('/content/drive/MyDrive/TFM/gru/gru_model_output')
# Directorio donde guardaremos los resultados de la evaluación
EVALUATION_OUTPUT_DIR = os.path.join('/content/drive/MyDrive/TFM/gru/gru_evaluation_output')
os.makedirs(EVALUATION_OUTPUT_DIR, exist_ok=True)

# --- Archivos de Entrada ---
# Modelo entrenado
MODEL_FILE = os.path.join(MODEL_OUTPUT_DIR, 'best_gru_model.keras')
# Datos de Test preprocesados
TEST_DATA_FILE = os.path.join(DATA_PREP_DIR, 'gru_test_data.npz')
# Scaler ajustado
SCALER_FILE = os.path.join(DATA_PREP_DIR, 'gru_scaler.joblib')
# Archivo Parquet ORIGINAL (necesario para recalcular y_train original para MASE)
# Asegúrate que esta ruta es correcta (la misma usada en el script de preparación)

PARQUET_FILE = '/content/drive/MyDrive/TFM/data/gold_ventas_semanales_training_clustered.parquet' # O la ruta directa
# Mapeos (necesario si recargamos y necesitamos mapear cluster_label de nuevo)
MAPPINGS_FILE = os.path.join(DATA_PREP_DIR, 'gru_cat_mappings.json')


# --- Archivos de Salida ---
METRICS_FILE = os.path.join(EVALUATION_OUTPUT_DIR, 'evaluation_metrics.json')
PREDICTION_PLOT_FILE = os.path.join(EVALUATION_OUTPUT_DIR, 'predictions_vs_actuals_scatter.png')

# --- Constantes ---
TARGET_COL = 'weekly_volume'
TIME_COL = 'week'
# Columnas numéricas que se escalaron juntas
NUMERICAL_FEATURES_SCALED = ['weekly_volume', 'has_promo', 'is_covid_period']


# --- 2. Cargar Modelo, Datos de Test y Scaler ---
print("Cargando modelo, datos de test y scaler...")
try:
    # Cargar modelo
    model = tf.keras.models.load_model(MODEL_FILE)
    print(f"Modelo cargado desde: {MODEL_FILE}")

    # Cargar datos de test preprocesados (.npz)
    test_data = np.load(TEST_DATA_FILE)
    X_test_num = test_data['X_num']
    y_test_scaled = test_data['y']
    # Extraer inputs categóricos de test
    X_test_cat_estab = test_data['establecimiento_mapped']
    X_test_cat_material = test_data['material_mapped']
    X_test_cat_cluster = test_data['cluster_label_mapped']
    print("Datos de test (.npz) cargados.")

    # Cargar scaler
    scaler = joblib.load(SCALER_FILE)
    print(f"Scaler cargado desde: {SCALER_FILE}")

    # Preparar inputs para predicción
    X_test_inputs = [X_test_num, X_test_cat_estab, X_test_cat_material, X_test_cat_cluster]

except Exception as e:
    print(f"Error cargando archivos necesarios: {e}")
    exit()


# --- 3. Realizar Predicciones ---
print("Realizando predicciones en el conjunto de test...")
y_pred_scaled = model.predict(X_test_inputs)
print(f"Predicciones realizadas. Shape: {y_pred_scaled.shape}")


# --- 4. Invertir Escalado (Desescalar) ---
print("Invirtiendo escalado de predicciones y valores reales...")

def inverse_scale_target(scaled_values, scaler_object, target_col_index, num_features_scaled):
    """Invierte el escalado solo para la columna objetivo."""
    # Crear un array temporal con la forma correcta (n_samples, n_features_scaled)
    temp_array = np.zeros((len(scaled_values), num_features_scaled))
    # Poner los valores escalados en la columna correcta
    temp_array[:, target_col_index] = scaled_values.flatten()
    # Aplicar inverse_transform
    original_scale_array = scaler_object.inverse_transform(temp_array)
    # Extraer solo la columna objetivo desescalada
    original_target_values = original_scale_array[:, target_col_index]
    return original_target_values

# Encontrar el índice de la columna TARGET_COL en la lista de features escaladas
try:
    target_index = NUMERICAL_FEATURES_SCALED.index(TARGET_COL)
    print(f"Índice de '{TARGET_COL}' para desescalado: {target_index}")
except ValueError:
    print(f"Error: '{TARGET_COL}' no encontrada en NUMERICAL_FEATURES_SCALED: {NUMERICAL_FEATURES_SCALED}")
    exit()

# Número de features que el scaler espera
n_features_for_scaler = len(NUMERICAL_FEATURES_SCALED)

# Desescalar predicciones
y_pred_original = inverse_scale_target(y_pred_scaled, scaler, target_index, n_features_for_scaler)

# Desescalar valores reales de test
y_test_original = inverse_scale_target(y_test_scaled, scaler, target_index, n_features_for_scaler)

print("Desescalado completado.")


# --- 5. Calcular Métricas de Evaluación ---
print("Calculando métricas de evaluación...")

mae = mean_absolute_error(y_test_original, y_pred_original)
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
r2 = r2_score(y_test_original, y_pred_original)

# Calcular MAPE con precaución por posibles ceros
# Reemplazar infinitos por NaN si ocurren
y_test_no_zeros = y_test_original.copy()
y_test_no_zeros[y_test_no_zeros == 0] = 1e-9 # Evitar división por cero
mape = mean_absolute_percentage_error(y_test_no_zeros, y_pred_original) * 100 # En porcentaje

print(f"  MAE: {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R2: {r2:.4f}")
print(f"  MAPE: {mape:.2f}%")


# --- 5.1 Calcular MASE ---
print("Calculando MASE...")
try:
    # Para MASE, necesitamos y_train original (no escalado)
    # Recargar datos originales y filtrar por fechas de entrenamiento
    print("  Recargando datos originales para obtener y_train...")
    df_full = pd.read_parquet(PARQUET_FILE, columns=[TIME_COL, TARGET_COL])
    df_full[TIME_COL] = pd.to_datetime(df_full[TIME_COL])

    # Necesitamos las fechas de corte usadas en la preparación
    unique_dates_full = np.sort(df_full[TIME_COL].unique())
    n_dates_full = len(unique_dates_full)
    n_test_dates_full = int(n_dates_full * TEST_SIZE_RATIO) # Usar mismos ratios
    n_val_dates_full = int(n_dates_full * VAL_SIZE_RATIO)
    train_cutoff_date = unique_dates_full[n_dates_full - n_test_dates_full - n_val_dates_full]

    # Filtrar para obtener solo y_train original
    y_train_original_series = df_full[df_full[TIME_COL] < train_cutoff_date][TARGET_COL]
    y_train_original = y_train_original_series.values
    del df_full # Liberar memoria
    gc.collect()

    if len(y_train_original) < 2:
        print("  y_train original demasiado corto para calcular MASE.")
        mase = np.nan
    else:
        # Calcular error absoluto medio del pronóstico naive (diferencia anterior) en train
        # Usamos np.diff que calcula la diferencia entre elementos consecutivos
        mean_abs_diff_train = np.mean(np.abs(np.diff(y_train_original)))

        if mean_abs_diff_train < 1e-9: # Evitar división por cero si la serie de train es constante
            print("  Error Naive en train es casi cero. MASE no se puede calcular o es Inf.")
            mase = np.inf if mae > 1e-9 else 0.0
        else:
            # MASE = MAE (test) / Mean Absolute Naive Error (train)
            mase = mae / mean_abs_diff_train
    print(f"  MASE: {mase:.4f}")

except Exception as e:
    print(f"  Error calculando MASE: {e}. Se asignará NaN.")
    mase = np.nan

# --- 6. Guardar y Mostrar Resultados ---
metrics = {
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2,
    'MAPE (%)': mape,
    'MASE': mase
}

print("\n--- Resultados de Evaluación (Conjunto de Test) ---")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}") # Para NaNs o Infs

# Guardar métricas en JSON
with open(METRICS_FILE, 'w') as f:
    # Convertir NaNs/Infs a strings para JSON si es necesario, o manejar antes
    serializable_metrics = {k: (str(v) if pd.isna(v) or np.isinf(v) else v) for k, v in metrics.items()}
    json.dump(serializable_metrics, f, indent=4)
print(f"\nMétricas guardadas en: {METRICS_FILE}")


# --- 7. (Opcional) Visualizar Predicciones vs Actuales ---
print("\nGenerando gráfico scatter de Predicciones vs Actuales...")
try:
    plt.figure(figsize=(10, 10))
    plt.scatter(y_test_original, y_pred_original, alpha=0.5, s=10) # 's' para tamaño de punto
    # Línea ideal y=x
    min_val = min(y_test_original.min(), y_pred_original.min())
    max_val = max(y_test_original.max(), y_pred_original.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Ideal (Predicción = Real)')
    plt.xlabel("Valores Reales (Original Scale)")
    plt.ylabel("Predicciones (Original Scale)")
    plt.title("Predicciones vs Valores Reales en Conjunto de Test")
    plt.grid(True)
    plt.legend()
    plt.axis('equal') # Asegura que la escala sea la misma en ambos ejes
    plt.tight_layout()
    plt.savefig(PREDICTION_PLOT_FILE)
    plt.close()
    print(f"Gráfico scatter guardado en: {PREDICTION_PLOT_FILE}")
except Exception as e:
    print(f"Error generando gráfico scatter: {e}")


print("\n--- Evaluación del modelo completada ---")

NameError: name 'N_TIMESTEPS_STR' is not defined

In [ ]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split # Lo usaremos solo como referencia, haremos split manual
import joblib # Para guardar el scaler y los mapeos
import os
import gc
import json # Para guardar mapeos

# --- 1. Configuración ---

# Asegúrate de que esta ruta apunte al archivo Parquet ORIGINAL (sin features de lag/roll)
PARQUET_FILE = '/content/drive/MyDrive/TFM/data/gold_venta_semanales_clusters.parquet' # O la ruta directa a tu archivo parquet original

# Directorio de salida para datos preprocesados y objetos
OUTPUT_DIR_GRU_PREP = os.path.join('/content/drive/MyDrive/TFM/gru/gru_data_preparation_output')
os.makedirs(OUTPUT_DIR_GRU_PREP, exist_ok=True)

# Archivos de salida
SCALER_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_scaler.joblib')
MAPPINGS_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_cat_mappings.json')
# Guardaremos los datos procesados como NumPy arrays
TRAIN_DATA_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_train_data.npz')
VAL_DATA_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_val_data.npz')
TEST_DATA_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_test_data.npz')


# Parámetros
VAL_SIZE_RATIO = 0.15 # 15% para validación
TEST_SIZE_RATIO = 0.15 # 15% para test

# Columnas a usar
TARGET_COL = 'weekly_volume'
TIME_COL = 'week'
SERIES_ID_COLS = ['establecimiento', 'material']
# Features categóricas que necesitan mapeo y embedding
CATEGORICAL_FEATURES = ['establecimiento', 'material', 'cluster_label']
# Features numéricas (incluyendo target para escalado y uso como input lag)
NUMERICAL_FEATURES = ['weekly_volume', 'has_promo', 'is_covid_period']


# --- 2. Carga y Ordenación de Datos ---
print("Cargando datos...")
try:
    # Cargar solo las columnas necesarias para eficiencia
    all_needed_cols = SERIES_ID_COLS + [TIME_COL, TARGET_COL] + \
                      [f for f in NUMERICAL_FEATURES if f != TARGET_COL] + \
                      [f for f in CATEGORICAL_FEATURES if f not in SERIES_ID_COLS]

    # Asegurar que no haya duplicados en la lista de columnas
    all_needed_cols = sorted(list(set(all_needed_cols)))

    df = pd.read_parquet(PARQUET_FILE, columns=all_needed_cols)
    print(f"Datos cargados: {df.shape}")
    print(f"Columnas cargadas: {df.columns.tolist()}")

    # Convertir columna de fecha si no lo está
    df[TIME_COL] = pd.to_datetime(df[TIME_COL])

    # Ordenar estrictamente por serie y tiempo
    df = df.sort_values(by=SERIES_ID_COLS + [TIME_COL]).reset_index(drop=True)
    print("Datos ordenados.")

except Exception as e:
    print(f"Error cargando o procesando el archivo Parquet: {e}")
    exit()

# --- 3. Mapeo de Features Categóricas a Enteros ---
print("Mapeando features categóricas...")
mappings = {}
vocab_sizes = {}
print(df)
for col in CATEGORICAL_FEATURES:
    unique_values = df[col].unique()
    # Crear mapeo: valor -> entero (0-based)
    cat_map = {val: i for i, val in enumerate(unique_values)}
    # Aplicar mapeo
    df[f'{col}_mapped'] = df[col].map(cat_map)
    # Guardar mapeo y tamaño de vocabulario
    mappings[col] = cat_map
    vocab_sizes[col] = len(unique_values)
    print(f"  - Mapeada '{col}': {len(unique_values)} categorías únicas.")

# Guardar los mapeos
with open(MAPPINGS_FILE, 'w') as f:
    # Convertir mapeos a formato serializable (string keys si es necesario)
     serializable_mappings = {k: {str(v_k): v_v for v_k, v_v in v.items()} for k, v in mappings.items()}
     json.dump({'mappings': serializable_mappings, 'vocab_sizes': vocab_sizes}, f, indent=4)
print(f"Mapeos guardados en: {MAPPINGS_FILE}")

# Columnas mapeadas que usaremos
MAPPED_CAT_COLS = [f'{col}_mapped' for col in CATEGORICAL_FEATURES]


# --- 4. División Train/Validation/Test (Cronológica Global) ---
print("Realizando división Train/Validation/Test cronológica...")
unique_dates = df[TIME_COL].unique()
unique_dates = np.sort(unique_dates)

n_dates = len(unique_dates)
n_test_dates = int(n_dates * TEST_SIZE_RATIO)
n_val_dates = int(n_dates * VAL_SIZE_RATIO)
n_train_dates = n_dates - n_test_dates - n_val_dates

# Determinar fechas de corte
test_cutoff_date = unique_dates[n_train_dates + n_val_dates]
val_cutoff_date = unique_dates[n_train_dates]

print(f"Total de fechas únicas: {n_dates}")
print(f"Fecha corte Train/Validation: {val_cutoff_date}")
print(f"Fecha corte Validation/Test: {test_cutoff_date}")

# Dividir DataFrame
train_df = df[df[TIME_COL] < val_cutoff_date].copy()
val_df = df[(df[TIME_COL] >= val_cutoff_date) & (df[TIME_COL] < test_cutoff_date)].copy()
test_df = df[df[TIME_COL] >= test_cutoff_date].copy()

print(f"Tamaño Train: {train_df.shape}")
print(f"Tamaño Validation: {val_df.shape}")
print(f"Tamaño Test: {test_df.shape}")

# Liberar memoria
original_df_memory = df.memory_usage(deep=True).sum() / (1024**2)
del df
gc.collect()
print(f"DataFrame original liberado (aprox {original_df_memory:.2f} MB).")


# --- 5. Escalado (MinMaxScaler) ---
print("Escalando features numéricas...")
scaler = MinMaxScaler(feature_range=(0, 1))
# Ojo: `NUMERICAL_FEATURES` incluye el target `weekly_volume`
numerical_cols_to_scale = NUMERICAL_FEATURES

# Ajustar scaler SOLO en datos de entrenamiento
scaler.fit(train_df[numerical_cols_to_scale])

# Guardar el scaler ajustado
joblib.dump(scaler, SCALER_FILE)
print(f"Scaler guardado en: {SCALER_FILE}")

# Aplicar transformación a todos los conjuntos
train_df[numerical_cols_to_scale] = scaler.transform(train_df[numerical_cols_to_scale])
val_df[numerical_cols_to_scale] = scaler.transform(val_df[numerical_cols_to_scale])
test_df[numerical_cols_to_scale] = scaler.transform(test_df[numerical_cols_to_scale])
print("Escalado aplicado a Train, Validation y Test.")


# --- 6. Generación de Secuencias ---
print("Generando secuencias...")

def create_sequences(df_input, n_timesteps, num_feature_cols, cat_feature_cols, target_col):
    """
    Genera secuencias para un modelo RNN/GRU a partir de un DataFrame.
    Asume que df_input está ordenado por serie y tiempo.
    Devuelve múltiples arrays NumPy para los inputs y un array para el target.
    """
    # Listas para almacenar los resultados
    X_num_list = []
    # Crear una lista de listas para los inputs categóricos
    X_cat_lists = {cat_col: [] for cat_col in cat_feature_cols}
    y_list = []

    # Agrupar por identificador de serie
    grouped = df_input.groupby(SERIES_ID_COLS)

    total_series = len(grouped)
    processed_series = 0

    for _, group in grouped:
        processed_series += 1
        if processed_series % 100 == 0: # Imprimir progreso cada 100 series
             print(f"  Procesando serie {processed_series}/{total_series}...")

        # Convertir a NumPy para eficiencia
        group_num_data = group[num_feature_cols].values
        group_cat_data = {cat_col: group[f'{cat_col}_mapped'].values for cat_col in CATEGORICAL_FEATURES} # Usar las mapeadas
        group_target_data = group[target_col].values # El target ya está entre las numéricas y escalado

        n_rows = len(group)
        if n_rows < n_timesteps + 1: # Necesita al menos n_timesteps para input y 1 para output
            continue # Saltar series demasiado cortas

        # Iterar para crear secuencias
        for i in range(n_timesteps, n_rows):
            # Input numérico: ventana de n_timesteps que termina en i-1
            X_num_list.append(group_num_data[i-n_timesteps : i])

            # Input categórico: Tomar el valor del último paso de la ventana de input (i-1)
            for cat_col in CATEGORICAL_FEATURES:
                 X_cat_lists[f'{cat_col}_mapped'].append(group_cat_data[cat_col][i-1]) # <-- ID del último paso del input

            # Output: valor en el tiempo i
            y_list.append(group_target_data[i]) # <-- Target en el tiempo i

    print(f"  ...Procesamiento de {processed_series} series completado.")

    # Convertir listas a NumPy arrays
    final_X_num = np.array(X_num_list)
    final_y = np.array(y_list).reshape(-1, 1) # Asegurar shape (n_samples, 1)

    # Crear diccionario de arrays categóricos
    final_X_cat = {}
    for cat_col in CATEGORICAL_FEATURES:
        final_X_cat[f'{cat_col}_mapped'] = np.array(X_cat_lists[f'{cat_col}_mapped']).reshape(-1, 1) # Shape (n_samples, 1)

    return final_X_num, final_X_cat, final_y


# Aplicar la función a cada conjunto de datos
print("\nGenerando secuencias para Train...")
X_train_num, X_train_cat, y_train = create_sequences(train_df, N_TIMESTEPS, NUMERICAL_FEATURES, MAPPED_CAT_COLS, TARGET_COL)
print("\nGenerando secuencias para Validation...")
X_val_num, X_val_cat, y_val = create_sequences(val_df, N_TIMESTEPS, NUMERICAL_FEATURES, MAPPED_CAT_COLS, TARGET_COL)
print("\nGenerando secuencias para Test...")
X_test_num, X_test_cat, y_test = create_sequences(test_df, N_TIMESTEPS, NUMERICAL_FEATURES, MAPPED_CAT_COLS, TARGET_COL)

# Liberar memoria de DataFrames procesados
del train_df, val_df, test_df
gc.collect()

# Imprimir formas para verificar
print("\nFormas de los arrays generados:")
print(f"Train: X_num={X_train_num.shape}, X_cat={ {k:v.shape for k,v in X_train_cat.items()} }, y={y_train.shape}")
print(f"Validation: X_num={X_val_num.shape}, X_cat={ {k:v.shape for k,v in X_val_cat.items()} }, y={y_val.shape}")
print(f"Test: X_num={X_test_num.shape}, X_cat={ {k:v.shape for k,v in X_test_cat.items()} }, y={y_test.shape}")


# --- 7. Guardar Datos Procesados ---
print("\nGuardando datos procesados en formato npz...")

# Guardar Train
np.savez_compressed(TRAIN_DATA_FILE,
                    X_num=X_train_num,
                    y=y_train,
                    **X_train_cat) # Desempaquetar diccionario categórico
# Guardar Validation
np.savez_compressed(VAL_DATA_FILE,
                    X_num=X_val_num,
                    y=y_val,
                    **X_val_cat)
# Guardar Test
np.savez_compressed(TEST_DATA_FILE,
                    X_num=X_test_num,
                    y=y_test,
                    **X_test_cat)

print(f"Datos de Train guardados en: {TRAIN_DATA_FILE}")
print(f"Datos de Validation guardados en: {VAL_DATA_FILE}")
print(f"Datos de Test guardados en: {TEST_DATA_FILE}")

print("\n--- Preparación de datos para GRU completada ---")

# Puedes cargar los datos guardados así:
# data = np.load(TRAIN_DATA_FILE)
# X_train_num_loaded = data['X_num']
# y_train_loaded = data['y']
# X_train_cat_estab_loaded = data['establecimiento_mapped']
# ... y así para los demás y para val/test

Cargando datos...
Datos cargados: (93673306, 7)
Columnas cargadas: ['cluster_label', 'establecimiento', 'has_promo', 'is_covid_period', 'material', 'week', 'weekly_volume']
Datos ordenados.
Mapeando features categóricas...
          cluster_label establecimiento  has_promo  is_covid_period material  \
0                     0      8100000003          0                0   DL13LT   
1                     0      8100000003          0                0   DL13LT   
2                     0      8100000003          0                0   DL13LT   
3                     0      8100000003          0                0   DL13LT   
4                     0      8100000003          0                0   DL13LT   
...                 ...             ...        ...              ...      ...   
93673301              0      8100468715          0                1    FDT13   
93673302              0      8100468715          0                1    FDT13   
93673303              0      8100468715          0       

NameError: name 'N_TIMESTEPS' is not defined

In [ ]:
N_TIMESTEPS_EVAL = '52'
N_TIMESTEPS = 52

In [ ]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split # Lo usaremos solo como referencia, haremos split manual
import joblib # Para guardar el scaler y los mapeos
import os
import gc
import json # Para guardar mapeos
import tensorflow as tf

import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import joblib
import json
import os
import pandas as pd # Necesario para recargar datos originales para MASE
import matplotlib.pyplot as plt

# Asegúrate que los directorios y N_TIMESTEPS coincidan con el modelo a evaluar
N_TIMESTEPS_EVAL = 52 # O 4, o 52, el que corresponda a los datos/modelo
DATA_PREP_DIR = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_/35'
MODEL_OUTPUT_DIR =  '/content/drive/MyDrive/TFM/gru/gru_model_output_/35'
EVALUATION_OUTPUT_DIR = f'/content/drive/MyDrive/TFM/gru/gru_evaluation_output/35'
os.makedirs(EVALUATION_OUTPUT_DIR, exist_ok=True)

# --- Archivos de Entrada ---
MODEL_FILE = '/content/drive/MyDrive/TFM/gru/gru_model_output_/35/best_gru_model.keras'
TEST_DATA_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_/35/gru_test_data.npz'
SCALER_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_/35/gru_scaler.joblib'
MAPPINGS_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_/35/gru_cat_mappings.json'
PARQUET_FILE = '/content/drive/MyDrive/TFM/data/gold_ventas_semanales_training_clustered.parquet' # Necesario si quisiéramos MASE, pero no ahora

# --- Archivos de Salida ---
METRICS_GLOBAL_FILE = '/content/drive/MyDrive/TFM/gru/gru_evaluation_output/35/evaluation_metrics_global.json'
METRICS_PER_CLUSTER_FILE = '/content/drive/MyDrive/TFM/gru/gru_evaluation_output/35/evaluation_metrics_cluster.json'
PREDICTION_PLOT_FILE ='/content/drive/MyDrive/TFM/gru/gru_evaluation_output/35/predictions_vs_actuals_scatter.png'

# --- Constantes ---
TARGET_COL = 'weekly_volume'
TIME_COL = 'week'
NUMERICAL_FEATURES_SCALED = ['weekly_volume', 'has_promo', 'is_covid_period']
VAL_SIZE_RATIO = 0.15 # Necesario si quisiéramos MASE
TEST_SIZE_RATIO = 0.15 # Necesario si quisiéramos MASE


# --- 2. Cargar Modelo, Datos de Test, Scaler y Mapeos ---
print("Cargando modelo, datos de test, scaler y mapeos...")
try:
    model = tf.keras.models.load_model(MODEL_FILE)
    print(f"Modelo cargado desde: {MODEL_FILE}")

    test_data = np.load(TEST_DATA_FILE)
    X_test_num = test_data['X_num']
    y_test_scaled = test_data['y']
    X_test_cat_estab = test_data['establecimiento_mapped']
    X_test_cat_material = test_data['material_mapped']
    X_test_cat_cluster = test_data['cluster_label_mapped']
    print(f"Datos de test (.npz) cargados desde {TEST_DATA_FILE}.")

    scaler = joblib.load(SCALER_FILE)
    print(f"Scaler cargado desde: {SCALER_FILE}")

    # Cargar mapeos completos
    with open(MAPPINGS_FILE, 'r') as f:
        mappings_data = json.load(f)
        mappings = mappings_data['mappings']
        # Crear mapeo inverso para cluster_label (id -> label original)
        # Convertir keys (que eran ints pero se guardaron como str en JSON) de nuevo a int
        original_cluster_map = mappings.get('cluster_label', {})
        inv_cluster_map = {int(v): k for k, v in original_cluster_map.items()}

    print(f"Mapeos cargados desde: {MAPPINGS_FILE}")

    X_test_inputs = [X_test_num, X_test_cat_estab, X_test_cat_material, X_test_cat_cluster]

except Exception as e:
    print(f"Error cargando archivos necesarios: {e}")
    exit()


# --- 3. Realizar Predicciones ---
# (Sin cambios)
print("Realizando predicciones en el conjunto de test...")
y_pred_scaled = model.predict(X_test_inputs)
print(f"Predicciones realizadas. Shape: {y_pred_scaled.shape}")


# --- 4. Invertir Escalado (Desescalar) ---
# (Sin cambios)
print("Invirtiendo escalado de predicciones y valores reales...")
def inverse_scale_target(scaled_values, scaler_object, target_col_index, num_features_scaled):
    temp_array = np.zeros((len(scaled_values), num_features_scaled))
    temp_array[:, target_col_index] = scaled_values.flatten()
    original_scale_array = scaler_object.inverse_transform(temp_array)
    original_target_values = original_scale_array[:, target_col_index]
    return original_target_values
try:
    target_index = NUMERICAL_FEATURES_SCALED.index(TARGET_COL)
except ValueError:
    print(f"Error: '{TARGET_COL}' no encontrada en NUMERICAL_FEATURES_SCALED: {NUMERICAL_FEATURES_SCALED}")
    exit()
n_features_for_scaler = len(NUMERICAL_FEATURES_SCALED)
y_pred_original = inverse_scale_target(y_pred_scaled, scaler, target_index, n_features_for_scaler)
y_test_original = inverse_scale_target(y_test_scaled, scaler, target_index, n_features_for_scaler)
print("Desescalado completado.")


# --- 5. Calcular Métricas Globales ---
print("Calculando métricas globales...")
mae_global = mean_absolute_error(y_test_original, y_pred_original)
rmse_global = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
r2_global = r2_score(y_test_original, y_pred_original)

metrics_global = {'MAE': mae_global, 'RMSE': rmse_global, 'R2': r2_global}
print("\n--- Métricas Globales (Conjunto de Test) ---")
for key, value in metrics_global.items():
    print(f"  {key}: {value:.4f}")
with open(METRICS_GLOBAL_FILE, 'w') as f:
    json.dump(metrics_global, f, indent=4)
print(f"\nMétricas globales guardadas en: {METRICS_GLOBAL_FILE}")


# ===> NUEVA SECCIÓN: Calcular Métricas por Cluster <===
print("\nCalculando métricas por cluster (R2, MAE, RMSE)...")

results_per_cluster = []
unique_mapped_ids = np.unique(X_test_cat_cluster) # IDs numéricos presentes en test

print(f"Encontrados {len(unique_mapped_ids)} clusters únicos en los datos de test.")

for mapped_id in unique_mapped_ids:
    # Obtener etiqueta original del cluster (manejar posible error si ID no está en mapeo)
    original_label = inv_cluster_map.get(mapped_id, f"Unknown_ID_{mapped_id}")

    # Crear máscara para filtrar datos de este cluster
    mask = (X_test_cat_cluster.flatten() == mapped_id)

    # Filtrar valores reales y predicciones
    y_test_cluster = y_test_original[mask]
    y_pred_cluster = y_pred_original[mask]

    # Calcular métricas si hay suficientes datos
    if len(y_test_cluster) > 0:
        mae_cluster = mean_absolute_error(y_test_cluster, y_pred_cluster)
        rmse_cluster = np.sqrt(mean_squared_error(y_test_cluster, y_pred_cluster))
        # R2 requiere al menos 2 muestras para ser significativo
        r2_cluster = r2_score(y_test_cluster, y_pred_cluster) if len(y_test_cluster) > 1 else np.nan

        results_per_cluster.append({
            'Cluster': original_label,
            'Num_Samples': len(y_test_cluster),
            'MAE': mae_cluster,
            'RMSE': rmse_cluster,
            'R2': r2_cluster
        })
    else:
        print(f"  Cluster {original_label} (ID {mapped_id}) no tiene muestras en test después del filtrado.")

# Convertir resultados a DataFrame para mejor visualización y guardado
if results_per_cluster:
    metrics_df = pd.DataFrame(results_per_cluster)
    metrics_df = metrics_df.sort_values(by='Cluster').reset_index(drop=True)

    # Mostrar resultados en consola
    print("\n--- Métricas por Cluster (Conjunto de Test) ---")
    # Ajustar opciones de display de Pandas para ver bien los números
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    print(metrics_df.to_string()) # .to_string() para evitar truncamiento

    # Guardar en CSV
    metrics_df.to_csv(METRICS_PER_CLUSTER_FILE, index=False, float_format='%.4f')
    print(f"\nMétricas por cluster guardadas en: {METRICS_PER_CLUSTER_FILE}")
else:
    print("\nNo se pudieron calcular métricas por cluster (quizás no hay datos de test).")

# ===> FIN DE LA NUEVA SECCIÓN <===


# --- 7. (Opcional) Visualizar Predicciones vs Actuales ---
# (Sin cambios, genera el gráfico global)
print("\nGenerando gráfico scatter de Predicciones vs Actuales (Global)...")
# ... (código del gráfico igual que antes) ...
try:
    plt.figure(figsize=(10, 10))
    plt.scatter(y_test_original, y_pred_original, alpha=0.5, s=10)
    min_val = min(y_test_original.min(), y_pred_original.min()) * 0.95 # Ajustar límites
    max_val = max(y_test_original.max(), y_pred_original.max()) * 1.05
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Ideal (Predicción = Real)')
    plt.xlabel("Valores Reales (Original Scale)")
    plt.ylabel("Predicciones (Original Scale)")
    plt.title(f"Predicciones vs Valores Reales en Test (N_TIMESTEPS={N_TIMESTEPS_EVAL})")
    plt.grid(True)
    plt.legend()
    plt.xlim(min_val, max_val) # Establecer límites
    plt.ylim(min_val, max_val)
    plt.tight_layout()
    plt.savefig(PREDICTION_PLOT_FILE)
    plt.close()
    print(f"Gráfico scatter guardado en: {PREDICTION_PLOT_FILE}")
except Exception as e:
    print(f"Error generando gráfico scatter: {e}")


print(f"\n--- Evaluación del modelo (N_TIMESTEPS={N_TIMESTEPS_EVAL}) completada ---")

Cargando modelo, datos de test, scaler y mapeos...
Modelo cargado desde: /content/drive/MyDrive/TFM/gru/gru_model_output_/35/best_gru_model.keras
Datos de test (.npz) cargados desde /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_/35/gru_test_data.npz.
Scaler cargado desde: /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_/35/gru_scaler.joblib
Mapeos cargados desde: /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_/35/gru_cat_mappings.json
Realizando predicciones en el conjunto de test...
1212/1212 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
Predicciones realizadas. Shape: (38766, 1)
Invirtiendo escalado de predicciones y valores reales...
Desescalado completado.
Calculando métricas globales...

--- Métricas Globales (Conjunto de Test) ---
  MAE: 39.1443
  RMSE: 221.6045
  R2: 0.1721

Métricas globales guardadas en: /content/drive/MyDrive/TFM/gru/gru_evaluation_output/35/evaluation_metrics_global.json

Calculando métricas por cluster (R2, MAE, RMSE)...
Encontrados 7

In [ ]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split # Lo usaremos solo como referencia, haremos split manual
import joblib # Para guardar el scaler y los mapeos
import os
import gc
import json # Para guardar mapeos

# --- 1. Configuración ---
TARGET_CLUSTER_ID = 2

# Asegúrate de que esta ruta apunte al archivo Parquet ORIGINAL (sin features de lag/roll)
PARQUET_FILE = '/content/drive/MyDrive/TFM/data/gold_ventas_semanales_training_clustered.parquet' # O la ruta directa a tu archivo parquet original

# Directorio de salida para datos preprocesados y objetos
OUTPUT_DIR_GRU_PREP = os.path.join('/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster', TARGET_CLUSTER_ID)
os.makedirs(OUTPUT_DIR_GRU_PREP, exist_ok=True)

# Archivos de salida
SCALER_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_scaler.joblib')
MAPPINGS_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_cat_mappings.json')
# Guardaremos los datos procesados como NumPy arrays
TRAIN_DATA_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_train_data.npz')
VAL_DATA_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_val_data.npz')
TEST_DATA_FILE = os.path.join(OUTPUT_DIR_GRU_PREP, 'gru_test_data.npz')


# Parámetros
VAL_SIZE_RATIO = 0.25 # 15% para validación
TEST_SIZE_RATIO = 0.25 # 15% para test

# Columnas a usar
TARGET_COL = 'weekly_volume'
TIME_COL = 'week'
SERIES_ID_COLS = ['establecimiento', 'material']
# Features categóricas que necesitan mapeo y embedding
CATEGORICAL_FEATURES = ['establecimiento', 'material']
# Features numéricas (incluyendo target para escalado y uso como input lag)
NUMERICAL_FEATURES = ['weekly_volume', 'has_promo', 'is_covid_period']


# --- 2. Carga y Ordenación de Datos ---
print("Cargando datos...")
try:
    # Cargar solo las columnas necesarias para eficiencia
    all_needed_cols = SERIES_ID_COLS + [TIME_COL, TARGET_COL] + \
                      [f for f in NUMERICAL_FEATURES if f != TARGET_COL] + \
                      [f for f in CATEGORICAL_FEATURES if f not in SERIES_ID_COLS]

    # Asegurar que no haya duplicados en la lista de columnas
    all_needed_cols = sorted(list(set(all_needed_cols)))

    df_full = pd.read_parquet(PARQUET_FILE)
    print(f"Datos completos cargados: {df_full.shape}")

    # ===> FILTRAR POR CLUSTER <===
    df = df_full[df_full['cluster_label'] == TARGET_CLUSTER_ID].copy()
    print(df.shape)
    if df.empty:
        print(f"ERROR: No se encontraron datos para el cluster {TARGET_CLUSTER_ID}. Saliendo.")
        exit()
    print(f"Datos filtrados para Cluster {TARGET_CLUSTER_ID}: {df.shape}")
    del df_full # Liberar memoria
    gc.collect()

except Exception as e:
    print(f"Error cargando o procesando el archivo Parquet: {e}")
    exit()

# --- 3. Mapeo de Features Categóricas a Enteros ---
print("Mapeando features categóricas...")
mappings = {}
vocab_sizes = {}

for col in CATEGORICAL_FEATURES:
    unique_values = df[col].unique()
    # Crear mapeo: valor -> entero (0-based)
    cat_map = {val: i for i, val in enumerate(unique_values)}
    # Aplicar mapeo
    df[f'{col}_mapped'] = df[col].map(cat_map)
    # Guardar mapeo y tamaño de vocabulario
    mappings[col] = cat_map
    vocab_sizes[col] = len(unique_values)
    print(f"  - Mapeada '{col}': {len(unique_values)} categorías únicas.")

# Guardar los mapeos
with open(MAPPINGS_FILE, 'w') as f:
    # Convertir mapeos a formato serializable (string keys si es necesario)
     serializable_mappings = {k: {str(v_k): v_v for v_k, v_v in v.items()} for k, v in mappings.items()}
     json.dump({'mappings': serializable_mappings, 'vocab_sizes': vocab_sizes}, f, indent=4)
print(f"Mapeos guardados en: {MAPPINGS_FILE}")

# Columnas mapeadas que usaremos
MAPPED_CAT_COLS = [f'{col}_mapped' for col in CATEGORICAL_FEATURES]
print(df.columns)

# --- 4. División Train/Validation/Test (Cronológica Global) ---
print("Realizando división Train/Validation/Test cronológica...")
unique_dates = df[TIME_COL].unique()
unique_dates = np.sort(unique_dates)

n_dates = len(unique_dates)
n_test_dates = int(n_dates * TEST_SIZE_RATIO)
n_val_dates = int(n_dates * VAL_SIZE_RATIO)
n_train_dates = n_dates - n_test_dates - n_val_dates
if n_train_dates < 1 or n_val_dates < 1 or n_test_dates < 1: # Nuevo chequeo
   # ...
   exit()

# Determinar fechas de corte
test_cutoff_date = unique_dates[n_train_dates + n_val_dates]
val_cutoff_date = unique_dates[n_train_dates]

print(f"Total de fechas únicas: {n_dates}")
print(f"Fecha corte Train/Validation: {val_cutoff_date}")
print(f"Fecha corte Validation/Test: {test_cutoff_date}")

# Dividir DataFrame
train_df = df[df[TIME_COL] < val_cutoff_date].copy()
val_df = df[(df[TIME_COL] >= val_cutoff_date) & (df[TIME_COL] < test_cutoff_date)].copy()
test_df = df[df[TIME_COL] >= test_cutoff_date].copy()

print(f"Tamaño Train: {train_df.shape}")
print(f"Tamaño Validation: {val_df.shape}")
print(f"Tamaño Test: {test_df.shape}")

# Liberar memoria
original_df_memory = df.memory_usage(deep=True).sum() / (1024**2)
del df
gc.collect()
print(f"DataFrame original liberado (aprox {original_df_memory:.2f} MB).")


# --- 5. Escalado (MinMaxScaler) ---
print("Escalando features numéricas...")
scaler = MinMaxScaler(feature_range=(0, 1))
# Ojo: `NUMERICAL_FEATURES` incluye el target `weekly_volume`
numerical_cols_to_scale = NUMERICAL_FEATURES
if train_df.empty: # Nuevo chequeo
    # ...
    exit()
scaler.fit(train_df[numerical_cols_to_scale]) # Fit solo en train del cluster

# Guardar el scaler ajustado
joblib.dump(scaler, SCALER_FILE)
print(f"Scaler guardado en: {SCALER_FILE}")

# Aplicar transformación a todos los conjuntos
train_df[numerical_cols_to_scale] = scaler.transform(train_df[numerical_cols_to_scale])
val_df[numerical_cols_to_scale] = scaler.transform(val_df[numerical_cols_to_scale])
test_df[numerical_cols_to_scale] = scaler.transform(test_df[numerical_cols_to_scale])
print("Escalado aplicado a Train, Validation y Test.")


# --- 6. Generación de Secuencias ---
print("Generando secuencias...")

def create_sequences_per_cluster(df_input, n_timesteps, num_feature_cols, mapped_cat_cols_local, target_col):
    X_num_list = []
    # Ahora solo esperamos las keys de mapped_cat_cols_local (estab, material)
    X_cat_lists = {cat_col: [] for cat_col in mapped_cat_cols_local}
    y_list = []
    grouped = df_input.groupby(SERIES_ID_COLS) # Agrupar por serie dentro del cluster
    total_series = len(grouped)
    processed_series = 0

    for _, group in grouped:
        processed_series += 1
        #if processed_series % 50 == 0: # Progreso opcional
        #     print(f"  Procesando serie {processed_series}/{total_series}...")

        group_num_data = group[num_feature_cols].values
        # Obtener datos categóricos usando los nombres de columnas mapeadas locales
        group_cat_data = {cat_col: group[cat_col].values for cat_col in mapped_cat_cols_local}
        group_target_data = group[target_col].values

        n_rows = len(group)
        if n_rows < n_timesteps + 1:
            continue

        for i in range(n_timesteps, n_rows):
            X_num_list.append(group_num_data[i-n_timesteps : i])
            # Obtener IDs categóricos (estab, material) del último paso
            for cat_col in mapped_cat_cols_local:
                 X_cat_lists[cat_col].append(group_cat_data[cat_col][i-1])
            y_list.append(group_target_data[i])

    print(f"  ...Procesamiento de {processed_series} series completado.")
    final_X_num = np.array(X_num_list) if X_num_list else np.empty((0, n_timesteps, len(num_feature_cols)))
    final_y = np.array(y_list).reshape(-1, 1) if y_list else np.empty((0, 1))
    final_X_cat = {}
    for cat_col in mapped_cat_cols_local:
        final_X_cat[cat_col] = np.array(X_cat_lists[cat_col]).reshape(-1, 1) if X_cat_lists[cat_col] else np.empty((0, 1))

    # Verificar si se generaron secuencias
    if final_X_num.shape[0] == 0:
        print(f"ADVERTENCIA: No se generaron secuencias para este DataFrame (posiblemente debido a N_TIMESTEPS={n_timesteps} y longitudes de series).")


    return final_X_num, final_X_cat, final_y

# Aplicar la función a cada conjunto de datos
print("\nGenerando secuencias para Train...")
X_train_num, X_train_cat, y_train = create_sequences_per_cluster(train_df, N_TIMESTEPS, NUMERICAL_FEATURES, MAPPED_CAT_COLS, TARGET_COL)
print("\nGenerando secuencias para Validation...")
X_val_num, X_val_cat, y_val = create_sequences_per_cluster(val_df, N_TIMESTEPS, NUMERICAL_FEATURES, MAPPED_CAT_COLS, TARGET_COL)
print("\nGenerando secuencias para Test...")
X_test_num, X_test_cat, y_test = create_sequences_per_cluster(test_df, N_TIMESTEPS, NUMERICAL_FEATURES, MAPPED_CAT_COLS, TARGET_COL)


del train_df, val_df, test_df
gc.collect()

print("\nFormas de los arrays generados:")
print(f"Train: X_num={X_train_num.shape}, X_cat={ {k:v.shape for k,v in X_train_cat.items()} }, y={y_train.shape}")
print(f"Validation: X_num={X_val_num.shape}, X_cat={ {k:v.shape for k,v in X_val_cat.items()} }, y={y_val.shape}")
print(f"Test: X_num={X_test_num.shape}, X_cat={ {k:v.shape for k,v in X_test_cat.items()} }, y={y_test.shape}")


# --- 7. Guardar Datos Procesados ---
print("\nGuardando datos procesados en formato npz...")

# Guardar Train (solo si no está vacío)
if X_train_num.shape[0] > 0:
    np.savez_compressed(TRAIN_DATA_FILE, X_num=X_train_num, y=y_train, **X_train_cat)
    print(f"Datos de Train guardados en: {TRAIN_DATA_FILE}")
else: print("Datos de Train vacíos, no se guardaron.")
# Guardar Validation
if X_val_num.shape[0] > 0:
    np.savez_compressed(VAL_DATA_FILE, X_num=X_val_num, y=y_val, **X_val_cat)
    print(f"Datos de Validation guardados en: {VAL_DATA_FILE}")
else: print("Datos de Validation vacíos, no se guardaron.")
# Guardar Test
if X_test_num.shape[0] > 0:
    np.savez_compressed(TEST_DATA_FILE, X_num=X_test_num, y=y_test, **X_test_cat)
    print(f"Datos de Test guardados en: {TEST_DATA_FILE}")
else: print("Datos de Test vacíos, no se guardaron.")

print(f"\n--- Preparación de datos para CLUSTER {TARGET_CLUSTER_ID} completada ---")


Cargando datos...
Datos completos cargados: (28362826, 7)
(18405031, 7)
Datos filtrados para Cluster 2: (18405031, 7)
Mapeando features categóricas...
  - Mapeada 'establecimiento': 79094 categorías únicas.
  - Mapeada 'material': 63 categorías únicas.
Mapeos guardados en: /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26/gru_cat_mappings.json
Index(['establecimiento', 'material', 'week', 'has_promo', 'weekly_volume',
       'is_covid_period', 'cluster_label', 'establecimiento_mapped',
       'material_mapped'],
      dtype='object')
Realizando división Train/Validation/Test cronológica...
Total de fechas únicas: 144
Fecha corte Train/Validation: 2023-08-21
Fecha corte Validation/Test: 2024-04-29
Tamaño Train: (9301776, 9)
Tamaño Validation: (5100463, 9)
Tamaño Test: (4002792, 9)
DataFrame original liberado (aprox 3802.38 MB).
Escalando features numéricas...
Scaler guardado en: /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26/gru_scaler.joblib

In [ ]:
 # train_gru_per_cluster.py

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Concatenate, Flatten, Dropout,RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import joblib
import json
import os
import matplotlib.pyplot as plt

# --- 1. Configuración ---

# ===> PARÁMETRO CLAVE: Especifica para qué cluster quieres entrenar <===
TARGET_CLUSTER_ID = 0  # Cambia a 2 en la segunda ejecución
# ===> Especifica el N_TIMESTEPS usado en la preparación de datos <===
N_TIMESTEPS_TRAIN = 52 # O 26, o el valor que usaste para preparar datos

# Directorio donde guardamos los datos preprocesados (específico del cluster)
DATA_PREP_DIR = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster0'
# Directorio donde guardaremos el modelo entrenado (específico del cluster)
MODEL_OUTPUT_DIR = '/content/drive/MyDrive/TFM/gru/gru_data_model_output_cluster20'
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

# Archivos de entrada (específicos del cluster)
TRAIN_DATA_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster0/gru_train_data.npz'
VAL_DATA_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster0/gru_val_data.npz'
MAPPINGS_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output/gru_cat_mappings.json'

# Archivo de salida para el mejor modelo (específico del cluster)
BEST_MODEL_FILE = os.path.join(MODEL_OUTPUT_DIR, f'best_gru_model_c{TARGET_CLUSTER_ID}.keras')
HISTORY_PLOT_FILE = os.path.join(MODEL_OUTPUT_DIR, f'training_history_c{TARGET_CLUSTER_ID}.png')


# --- Hiperparámetros del Modelo (Iguales o ajustables) ---
EMBEDDING_DIM_ESTAB = 10
EMBEDDING_DIM_MATERIAL = 5
# Ya no necesitamos EMBEDDING_DIM_CLUSTER
GRU_UNITS = 64
DROPOUT_RATE = 0.2
DENSE_UNITS = 32

# --- Parámetros de Entrenamiento (Iguales o ajustables) ---
EPOCHS = 100
BATCH_SIZE = 64
PATIENCE_EARLY_STOPPING = 10
PATIENCE_REDUCE_LR = 5


# --- 2. Cargar Datos Preprocesados y Mapeos (Específicos del Cluster) ---
print(f"--- Entrenando modelo para CLUSTER {TARGET_CLUSTER_ID} ---")
print(f"Cargando datos preprocesados y mapeos desde: {DATA_PREP_DIR}")

with open(MAPPINGS_FILE, 'r') as f:
    mappings_data = json.load(f)
    vocab_sizes = mappings_data['vocab_sizes'] # Ahora solo tendrá estab y material
    # Asegurarse de que las keys esperadas existen
    if 'establecimiento' not in vocab_sizes or 'material' not in vocab_sizes:
          raise ValueError("El archivo de mapeos no contiene 'establecimiento' y 'material'.")

train_data = np.load(TRAIN_DATA_FILE)
X_train_num = train_data['X_num']
y_train = train_data['y']
X_train_cat_estab = train_data['establecimiento_mapped']
X_train_cat_material = train_data['material_mapped']
# Ya no cargamos X_train_cat_cluster

val_data = np.load(VAL_DATA_FILE)
X_val_num = val_data['X_num']
y_val = val_data['y']
X_val_cat_estab = val_data['establecimiento_mapped']
X_val_cat_material = val_data['material_mapped']
# Ya no cargamos X_val_cat_cluster

n_samples_train, n_timesteps_loaded, n_numerical_features = X_train_num.shape
n_samples_val = X_val_num.shape[0]

if n_timesteps_loaded != N_TIMESTEPS_TRAIN:
    raise ValueError(f"N_TIMESTEPS_TRAIN ({N_TIMESTEPS_TRAIN}) no coincide con datos cargados ({n_timesteps_loaded})")

# Chequeo rápido si hay datos suficientes para entrenar/validar
if n_samples_train == 0:
    print(f"ERROR: No hay datos de entrenamiento para el cluster {TARGET_CLUSTER_ID}. Saliendo.")
    exit()
if n_samples_val == 0:
    print(f"ADVERTENCIA: No hay datos de validación para el cluster {TARGET_CLUSTER_ID}. El entrenamiento procederá sin validación.")
    # Si no hay datos de validación, no podemos usar callbacks que dependan de ellos
    X_val_inputs = None
    y_val_for_fit = None
    validation_data_arg = None
else:
    # Preparar listas de inputs (SIN cluster)
    X_train_inputs = [X_train_num, X_train_cat_estab, X_train_cat_material]
    X_val_inputs = [X_val_num, X_val_cat_estab, X_val_cat_material]
    y_val_for_fit = y_val
    validation_data_arg = (X_val_inputs, y_val_for_fit)


print(f"Datos de entrenamiento cargados (Cluster {TARGET_CLUSTER_ID}): X_num={X_train_num.shape}, y={y_train.shape}")
print(f"  Inputs categóricos train shapes: estab={X_train_cat_estab.shape}, material={X_train_cat_material.shape}")
if n_samples_val > 0:
  print(f"Datos de validación cargados (Cluster {TARGET_CLUSTER_ID}): X_num={X_val_num.shape}, y={y_val.shape}")
print(f"Tamaños de Vocabulario (Cluster {TARGET_CLUSTER_ID}): {vocab_sizes}")

# --- 3. Construir el Modelo GRU (Modificado: Sin Cluster Input/Embedding) ---
print("\nConstruyendo el modelo GRU (específico del cluster)...")

# ===> FUNCIÓN MODIFICADA <===
def build_gru_model_per_cluster(n_timesteps, n_num_features, vocab_sizes_dict, emb_dims_dict, gru_units, dropout_rate, dense_units=None):
    """Construye el modelo GRU para un cluster (sin input de cluster)."""

    # Inputs (Numérico, Establecimiento, Material)
    input_numerical = Input(shape=(n_timesteps, n_num_features), name='numerical_input')
    input_cat_estab = Input(shape=(1,), name='establecimiento_input', dtype='int32')
    input_cat_material = Input(shape=(1,), name='material_input', dtype='int32')
    # No hay input_cat_cluster

    # Embeddings (Establecimiento, Material)
    emb_estab = Embedding(input_dim=vocab_sizes_dict['establecimiento'], output_dim=5, name='establecimiento_embedding')(input_cat_estab)
    emb_material = Embedding(input_dim=vocab_sizes_dict['material'], output_dim=5, name='material_embedding')(input_cat_material)

    estab_flat = Flatten()(emb_estab)
    mat_flat = Flatten()(emb_material)

    # No hay emb_cluster
    categorical_combined = Concatenate()([estab_flat, mat_flat]) # Note the extra () after Concatenate
    categorical_repeated = RepeatVector(26)(categorical_combined) # Now you pass the OUTPUT TENSOR
    #flat_emb_estab = Flatten()(emb_estab)
    #flat_emb_material = Flatten()(emb_material)
    # No hay flat_emb_cluster

    # GRU (igual que antes)
    gru_input = Concatenate(axis=-1)([input_numerical, categorical_repeated]) # Concatenate along the feature axis

    gru_layer = GRU(units=gru_units, name='gru_layer')(gru_input)
    gru_layer = Dropout(dropout_rate)(gru_layer)

    # Concatenate (Solo GRU, Estab, Material)
    #combined = Concatenate(name='concatenate_features')([gru_layer, flat_emb_estab, flat_emb_material])
    hidden = Dense(32, activation='relu', name='hidden_dense')(gru_layer)
    output = Dense(1, name='output')(hidden)

    # Dense layers (igual que antes)
    #x = combined
    #  x = Dense(dense_units, activation='relu', name='hidden_dense')(x)
    #if dense_units:
    #output = Dense(1, activation='linear', name='output')(x)

    # Crear Modelo (Inputs modificados)
    model = Model(inputs=[input_numerical, input_cat_estab, input_cat_material], outputs=output)
    return model
# ===> FIN FUNCIÓN MODIFICADA <===

# Crear instancia del modelo modificado
model = build_gru_model_per_cluster(
    n_timesteps=N_TIMESTEPS_TRAIN,
    n_num_features=n_numerical_features,
    vocab_sizes_dict=vocab_sizes, # Solo contiene estab y material
    emb_dims_dict={
        'establecimiento': EMBEDDING_DIM_ESTAB,
        'material': EMBEDDING_DIM_MATERIAL
        # No se pasa cluster_label
    },
    gru_units=GRU_UNITS,
    dropout_rate=DROPOUT_RATE,
    dense_units=DENSE_UNITS
)

model.summary()


# --- 4. Compilar el Modelo ---
# (Sin cambios)
print("\nCompilando el modelo...")
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


# --- 5. Configurar Callbacks ---
# (Ligeramente modificado para manejar sin validación)
print("Configurando callbacks...")
callbacks_list = []
monitor_metric = 'val_loss' if n_samples_val > 0 else 'loss' # Monitorear 'loss' si no hay validación

early_stopping = EarlyStopping(
    monitor=monitor_metric,
    patience=PATIENCE_EARLY_STOPPING,
    restore_best_weights=True
)
callbacks_list.append(early_stopping)

model_checkpoint = ModelCheckpoint(
    filepath=BEST_MODEL_FILE,
    monitor=monitor_metric,
    save_best_only=True,
    save_weights_only=False
)
callbacks_list.append(model_checkpoint)

if n_samples_val > 0: # Solo añadir ReduceLROnPlateau si hay validación
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=PATIENCE_REDUCE_LR,
        min_lr=1e-6
    )
    callbacks_list.append(reduce_lr)


# --- 6. Entrenar el Modelo ---
print(f"\nIniciando entrenamiento del modelo para Cluster {TARGET_CLUSTER_ID}...")

# ===> INPUTS MODIFICADOS PARA model.fit <===
history = model.fit(
    X_train_inputs, # Ya no contiene el ID de cluster
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=validation_data_arg, # Será None si no hay datos de validación
    callbacks=callbacks_list,
    verbose=1
)

print(f"\n--- Entrenamiento completado para Cluster {TARGET_CLUSTER_ID} ---")
# Si el entrenamiento se completó, el mejor modelo (si hubo validación)
# o el último (si no hubo validación pero se guardó) está en BEST_MODEL_FILE
# Nota: Si no hubo validación, ModelCheckpoint guardó el último. EarlyStopping basado en 'loss' puede ser menos útil.
print(f"Modelo guardado en: {BEST_MODEL_FILE}")


# --- 7. (Opcional) Visualizar Historial de Entrenamiento ---
print("\nGenerando gráfico de historial de entrenamiento...")
# (Misma función plot_training_history que antes, pero puede fallar si no hay 'val_loss'/'val_mae')
def plot_training_history(history, output_filename, has_validation):
    """Genera gráficos de pérdida y métricas de entrenamiento/validación."""
    try:
        fig, ax1 = plt.subplots(figsize=(12, 6))
        color = 'tab:red'
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss (MSE)', color=color)
        ax1.plot(history.history['loss'], label='Train Loss', color=color, linestyle='-')
        if has_validation:
           ax1.plot(history.history['val_loss'], label='Validation Loss', color=color, linestyle='--')
        ax1.tick_params(axis='y', labelcolor=color)
        ax1.legend(loc='upper left')
        ax1.grid(True)

        if 'mae' in history.history: # Chequear si MAE fue compilado
            ax2 = ax1.twinx()
            color = 'tab:blue'
            ax2.set_ylabel('MAE', color=color)
            ax2.plot(history.history['mae'], label='Train MAE', color=color, linestyle='-')
            if has_validation and 'val_mae' in history.history:
               ax2.plot(history.history['val_mae'], label='Validation MAE', color=color, linestyle='--')
            ax2.tick_params(axis='y', labelcolor=color)
            ax2.legend(loc='upper right')

        fig.tight_layout()
        plt.title(f'Training History - Cluster {TARGET_CLUSTER_ID}')
        plt.savefig(output_filename)
        plt.close()
        print(f"Gráfico de historial guardado en: {output_filename}")
    except Exception as e:
        print(f"Error al generar gráfico de historial: {e}")

plot_training_history(history, HISTORY_PLOT_FILE, n_samples_val > 0)

print(f"\nScript de entrenamiento para Cluster {TARGET_CLUSTER_ID} finalizado.")

--- Entrenando modelo para CLUSTER 0 ---
Cargando datos preprocesados y mapeos desde: /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster0


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster0/gru_cat_mappings.json'

In [ ]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split # Lo usaremos solo como referencia, haremos split manual
import joblib # Para guardar el scaler y los mapeos
import os
import gc
import json # Para guardar mapeos
import tensorflow as tf

import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import joblib
import json
import os
import pandas as pd # Necesario para recargar datos originales para MASE
import matplotlib.pyplot as plt

# Asegúrate que los directorios y N_TIMESTEPS coincidan con el modelo a evaluar
N_TIMESTEPS_EVAL = 26 # O 4, o 52, el que corresponda a los datos/modelo
DATA_PREP_DIR = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26'
MODEL_OUTPUT_DIR =  '/content/drive/MyDrive/TFM/gru/gru_data_model_output_cluster2'
EVALUATION_OUTPUT_DIR = f'/content/drive/MyDrive/TFM/gru/gru_evaluation_output_cluster2/26'
os.makedirs(EVALUATION_OUTPUT_DIR, exist_ok=True)

# --- Archivos de Entrada ---
MODEL_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_model_output_cluster2/best_gru_model_c2.keras'
TEST_DATA_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26/gru_test_data.npz'
SCALER_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26/gru_scaler.joblib'
MAPPINGS_FILE = '/content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26/gru_cat_mappings.json'
PARQUET_FILE = '/content/drive/MyDrive/TFM/data/gold_ventas_semanales_training_clustered.parquet' # Necesario si quisiéramos MASE, pero no ahora

# --- Archivos de Salida ---
METRICS_GLOBAL_FILE = '/content/drive/MyDrive/TFM/gru/gru_evaluation_output_cluster2/26/evaluation_metrics_global.json'
METRICS_PER_CLUSTER_FILE = '/content/drive/MyDrive/TFM/gru/gru_evaluation_output_cluster2/26/evaluation_metrics_cluster.json'
PREDICTION_PLOT_FILE ='/content/drive/MyDrive/TFM/gru/gru_evaluation_output_cluster2/26/predictions_vs_actuals_scatter.png'

# --- Constantes ---
TARGET_COL = 'weekly_volume'
TIME_COL = 'week'
NUMERICAL_FEATURES_SCALED = ['weekly_volume', 'has_promo', 'is_covid_period']
VAL_SIZE_RATIO = 0.25 # Necesario si quisiéramos MASE
TEST_SIZE_RATIO = 0.25 # Necesario si quisiéramos MASE


# --- 2. Cargar Modelo, Datos de Test, Scaler y Mapeos ---
print("Cargando modelo, datos de test, scaler y mapeos...")
try:
    model = tf.keras.models.load_model(MODEL_FILE)
    print(f"Modelo cargado desde: {MODEL_FILE}")

    test_data = np.load(TEST_DATA_FILE)
    X_test_num = test_data['X_num']
    y_test_scaled = test_data['y']
    X_test_cat_estab = test_data['establecimiento_mapped']
    X_test_cat_material = test_data['material_mapped']
    print(f"Datos de test (.npz) cargados desde {TEST_DATA_FILE}.")

    scaler = joblib.load(SCALER_FILE)
    print(f"Scaler cargado desde: {SCALER_FILE}")

    # Cargar mapeos completos
    with open(MAPPINGS_FILE, 'r') as f:
        mappings_data = json.load(f)
        mappings = mappings_data['mappings']
        # Crear mapeo inverso para cluster_label (id -> label original)
        # Convertir keys (que eran ints pero se guardaron como str en JSON) de nuevo a int
        original_cluster_map = mappings.get('cluster_label', {})
        inv_cluster_map = {int(v): k for k, v in original_cluster_map.items()}

    print(f"Mapeos cargados desde: {MAPPINGS_FILE}")

    X_test_inputs = [X_test_num, X_test_cat_estab, X_test_cat_material]

except Exception as e:
    print(f"Error cargando archivos necesarios: {e}")
    exit()


# --- 3. Realizar Predicciones ---
# (Sin cambios)
print("Realizando predicciones en el conjunto de test...")
y_pred_scaled = model.predict(X_test_inputs)
print(f"Predicciones realizadas. Shape: {y_pred_scaled.shape}")


# --- 4. Invertir Escalado (Desescalar) ---
# (Sin cambios)
print("Invirtiendo escalado de predicciones y valores reales...")
def inverse_scale_target(scaled_values, scaler_object, target_col_index, num_features_scaled):
    temp_array = np.zeros((len(scaled_values), num_features_scaled))
    temp_array[:, target_col_index] = scaled_values.flatten()
    original_scale_array = scaler_object.inverse_transform(temp_array)
    original_target_values = original_scale_array[:, target_col_index]
    return original_target_values
try:
    target_index = NUMERICAL_FEATURES_SCALED.index(TARGET_COL)
except ValueError:
    print(f"Error: '{TARGET_COL}' no encontrada en NUMERICAL_FEATURES_SCALED: {NUMERICAL_FEATURES_SCALED}")
    exit()
n_features_for_scaler = len(NUMERICAL_FEATURES_SCALED)
y_pred_original = inverse_scale_target(y_pred_scaled, scaler, target_index, n_features_for_scaler)
y_test_original = inverse_scale_target(y_test_scaled, scaler, target_index, n_features_for_scaler)
print("Desescalado completado.")


# --- 5. Calcular Métricas Globales ---
print("Calculando métricas globales...")
mae_global = mean_absolute_error(y_test_original, y_pred_original)
rmse_global = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
r2_global = r2_score(y_test_original, y_pred_original)

metrics_global = {'MAE': mae_global, 'RMSE': rmse_global, 'R2': r2_global}
print("\n--- Métricas Globales (Conjunto de Test) ---")
for key, value in metrics_global.items():
    print(f"  {key}: {value:.4f}")
with open(METRICS_GLOBAL_FILE, 'w') as f:
    json.dump(metrics_global, f, indent=4)
print(f"\nMétricas globales guardadas en: {METRICS_GLOBAL_FILE}")


# ===> NUEVA SECCIÓN: Calcular Métricas por Cluster <===
print("\nCalculando métricas por cluster (R2, MAE, RMSE)...")

results_per_cluster = []
unique_mapped_ids = np.unique(X_test_cat_cluster) # IDs numéricos presentes en test

print(f"Encontrados {len(unique_mapped_ids)} clusters únicos en los datos de test.")

for mapped_id in unique_mapped_ids:
    # Obtener etiqueta original del cluster (manejar posible error si ID no está en mapeo)
    original_label = inv_cluster_map.get(mapped_id, f"Unknown_ID_{mapped_id}")

    # Crear máscara para filtrar datos de este cluster
    mask = (X_test_cat_cluster.flatten() == mapped_id)

    # Filtrar valores reales y predicciones
    y_test_cluster = y_test_original[mask]
    y_pred_cluster = y_pred_original[mask]

    # Calcular métricas si hay suficientes datos
    if len(y_test_cluster) > 0:
        mae_cluster = mean_absolute_error(y_test_cluster, y_pred_cluster)
        rmse_cluster = np.sqrt(mean_squared_error(y_test_cluster, y_pred_cluster))
        # R2 requiere al menos 2 muestras para ser significativo
        r2_cluster = r2_score(y_test_cluster, y_pred_cluster) if len(y_test_cluster) > 1 else np.nan

        results_per_cluster.append({
            'Cluster': original_label,
            'Num_Samples': len(y_test_cluster),
            'MAE': mae_cluster,
            'RMSE': rmse_cluster,
            'R2': r2_cluster
        })
    else:
        print(f"  Cluster {original_label} (ID {mapped_id}) no tiene muestras en test después del filtrado.")

# Convertir resultados a DataFrame para mejor visualización y guardado
if results_per_cluster:
    metrics_df = pd.DataFrame(results_per_cluster)
    metrics_df = metrics_df.sort_values(by='Cluster').reset_index(drop=True)

    # Mostrar resultados en consola
    print("\n--- Métricas por Cluster (Conjunto de Test) ---")
    # Ajustar opciones de display de Pandas para ver bien los números
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    print(metrics_df.to_string()) # .to_string() para evitar truncamiento

    # Guardar en CSV
    metrics_df.to_csv(METRICS_PER_CLUSTER_FILE, index=False, float_format='%.4f')
    print(f"\nMétricas por cluster guardadas en: {METRICS_PER_CLUSTER_FILE}")
else:
    print("\nNo se pudieron calcular métricas por cluster (quizás no hay datos de test).")

# ===> FIN DE LA NUEVA SECCIÓN <===


# --- 7. (Opcional) Visualizar Predicciones vs Actuales ---
# (Sin cambios, genera el gráfico global)
print("\nGenerando gráfico scatter de Predicciones vs Actuales (Global)...")
# ... (código del gráfico igual que antes) ...
try:
    plt.figure(figsize=(10, 10))
    plt.scatter(y_test_original, y_pred_original, alpha=0.5, s=10)
    min_val = min(y_test_original.min(), y_pred_original.min()) * 0.95 # Ajustar límites
    max_val = max(y_test_original.max(), y_pred_original.max()) * 1.05
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Ideal (Predicción = Real)')
    plt.xlabel("Valores Reales (Original Scale)")
    plt.ylabel("Predicciones (Original Scale)")
    plt.title(f"Predicciones vs Valores Reales en Test (N_TIMESTEPS={N_TIMESTEPS_EVAL})")
    plt.grid(True)
    plt.legend()
    plt.xlim(min_val, max_val) # Establecer límites
    plt.ylim(min_val, max_val)
    plt.tight_layout()
    plt.savefig(PREDICTION_PLOT_FILE)
    plt.close()
    print(f"Gráfico scatter guardado en: {PREDICTION_PLOT_FILE}")
except Exception as e:
    print(f"Error generando gráfico scatter: {e}")


print(f"\n--- Evaluación del modelo (N_TIMESTEPS={N_TIMESTEPS_EVAL}) completada ---")

Cargando modelo, datos de test, scaler y mapeos...
Modelo cargado desde: /content/drive/MyDrive/TFM/gru/gru_data_model_output_cluster2/best_gru_model_c2.keras
Datos de test (.npz) cargados desde /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26/gru_test_data.npz.
Scaler cargado desde: /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26/gru_scaler.joblib
Mapeos cargados desde: /content/drive/MyDrive/TFM/gru/gru_data_preparation_output_cluster2/26/gru_cat_mappings.json
Realizando predicciones en el conjunto de test...
19073/19073 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step
Predicciones realizadas. Shape: (610314, 1)
Invirtiendo escalado de predicciones y valores reales...
Desescalado completado.
Calculando métricas globales...

--- Métricas Globales (Conjunto de Test) ---
  MAE: 10.4018
  RMSE: 27.2391
  R2: 0.1626

Métricas globales guardadas en: /content/drive/MyDrive/TFM/gru/gru_evaluation_output_cluster2/26/evaluation_metrics_global.json

Calculando métr

NameError: name 'X_test_cat_cluster' is not defined